<a href="https://colab.research.google.com/github/UrologyUnbound/SIOP_ML_2024_Discord/blob/main/colabs/Tune_Interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Interview Responses Model Fine Tuning

The goal of this notebook is to fine tune the OpenAI GPT 3.5 model to be able to generate plausible responses to interview questions.

# Inputs and Setup

In [ ]:
!pip install --upgrade tiktoken openai

In [2]:
import json
import openai
import os
import pandas as pd
from pprint import pprint
import tiktoken
from sklearn.model_selection import train_test_split
from google.colab import userdata
import numpy as np
from collections import defaultdict

client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
encoding = tiktoken.get_encoding("cl100k_base")

In [3]:
#Estimated token counter
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Data Import and Prep

In [5]:
df_interview = pd.read_csv("https://raw.githubusercontent.com/UrologyUnbound/SIOP_ML_2024_Discord/main/data/train/interview_train.csv")
df_interview.head()

,_id,questions_answers,last_answer
0,R_2q2WQ6Icq6pcuvZ,Question: Learning new skills keeps us competi...,"When motivating a team, I try to persuade them..."
1,R_3phEOOtqg43Bt0X,Question: Can you walk me through your decisio...,I use task prioritization and time blocking in...
2,R_3Olf80hXULyekYV,Question: How do you ensure that your skills s...,I attended a women in business panel conferenc...
3,R_1HdTINcsBSXfECD,Question: Describe a time when you felt overwh...,Contrast is criticism is something that I real...
4,R_1o4H0qHXqhhtHEd,Question: Can you walk me through your decisio...,At work one major change I had to face was wor...


In [6]:
x_train,x_test,y_train,y_test = train_test_split(df_interview["questions_answers"],df_interview["last_answer"],random_state=42,test_size = 0.2)
train_data = pd.concat([x_train , y_train], axis = 1)
test_data = pd.concat([x_train , y_train], axis = 1)
train_data.head()

,questions_answers,last_answer
8,Question: Can you walk me through the various ...,When I was faced with the decision of whether ...
16,Question: Share your strategies for maintainin...,It's important to understand that your coworke...
3,Question: Describe a time when you felt overwh...,Contrast is criticism is something that I real...
13,Question: Describe an occasion where you juggl...,"For one of my graduate-level classes, I had a ..."
15,Question: Share your strategies for maintainin...,As an overworked high schooler I often had to ...


In [8]:
system_message = """
Your task is to analyze a dataset of interview questions and responses. Based on the content, tone, and details of the provided answers, your task is to
 generate a plausible answer for the fourth interview question.
"""

def extract_questions_answers(text):
    # Split the text into parts based on "Question:" and "Response:" delimiters
    parts = text.split("Question:")[1:]
    qas = []

    for part in parts:
        q_a = part.split("Response:")
        question = q_a[0].strip()
        answer = q_a[1].strip() if len(q_a) > 1 else ""
        qas.append({"question": question, "answer": answer})

    # return qas
    return qas


def create_user_message(row):
    # Extract the questions and answers from the text
    qas = extract_questions_answers(row['questions_answers'])

    # Create the user message with the four questions and three answers
    user_message = ""
    for qa in qas:
        user_message += f"Question: {qa['question']}\n"
        if qa['answer']:
            user_message += f"Answer: {qa['answer']}\n"

    return user_message



def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    # Use the last_answer column as the assistant's message
    messages.append({"role": "assistant", "content": row["last_answer"]})

    return {"messages": messages}


pprint(prepare_example_conversation(train_data.iloc[0]))

{'messages': [{'content': '\n'
                          'Your task is to analyze a dataset of interview '
                          'questions and responses. Based on the content, '
                          'tone, and details of the provided answers, your '
                          'task is to \n'
                          ' generate a plausible answer for the fourth '
                          'interview question.\n',
               'role': 'system'},
              {'content': 'Question: Can you walk me through the various '
                          'strategies you employ when you find yourself under '
                          'a significant amount of stress, providing examples '
                          'from your past experience?\n'
                          'Answer: When I am under stress I try and do things '
                          'that will relieve it like going outside. Going to '
                          'the gym or going for a walk outside is very '
                

In [9]:
training_json = train_data.apply(prepare_example_conversation, axis=1).tolist()
test_json = test_data.apply(prepare_example_conversation, axis=1).tolist()


for example in training_json[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': '\nYour task is to analyze a dataset of interview questions and responses. Based on the content, tone, and details of the provided answers, your task is to \n generate a plausible answer for the fourth interview question.\n'}, {'role': 'user', 'content': 'Question: Can you walk me through the various strategies you employ when you find yourself under a significant amount of stress, providing examples from your past experience?\nAnswer: When I am under stress I try and do things that will relieve it like going outside. Going to the gym or going for a walk outside is very therapeutic and helpful when feeling overwhelmed. Being around those that I love also helps because I can confide in them and we together can come up with a solution. Asking for help is very important when under a significant amount of stress and I try to utilize this as often as I can. Dealing with stress is very tricky but using outlets like exercise and the comfort of frien

In [10]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [11]:
training_file_name = "tmp_interview_finetune_training.jsonl"
write_jsonl(training_json, training_file_name)

testing_file_name = "tmp_interview_finetune_testing.jsonl"
write_jsonl(test_json, testing_file_name)

In [13]:
!head -n 5 tmp_interview_finetune_training.jsonl

{"messages": [{"role": "system", "content": "\nYour task is to analyze a dataset of interview questions and responses. Based on the content, tone, and details of the provided answers, your task is to \n generate a plausible answer for the fourth interview question.\n"}, {"role": "user", "content": "Question: Can you walk me through the various strategies you employ when you find yourself under a significant amount of stress, providing examples from your past experience?\nAnswer: When I am under stress I try and do things that will relieve it like going outside. Going to the gym or going for a walk outside is very therapeutic and helpful when feeling overwhelmed. Being around those that I love also helps because I can confide in them and we together can come up with a solution. Asking for help is very important when under a significant amount of stress and I try to utilize this as often as I can. Dealing with stress is very tricky but using outlets like exercise and the comfort of frien

# Pre-Tuning Checks

In [14]:
# Format error checks - Training set
with open("/content/tmp_interview_finetune_testing.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [15]:
# Format error checks - Training set
with open("/content/tmp_interview_finetune_training.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [16]:
# Warnings and tokens counts
with open("/content/tmp_interview_finetune_testing.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 399, 633
mean / median: 502.8, 487.0
p5 / p95: 430.4, 601.8

#### Distribution of num_assistant_tokens_per_example:
min / max: 69, 175
mean / median: 95.26666666666667, 90.0
p5 / p95: 70.4, 123.39999999999999

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [17]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print(f"Estimated training cost ~${((n_epochs * n_billing_tokens_in_dataset)/1000)*.0080}")

Dataset has ~7542 tokens that will be charged for during training
By default, you'll train for 6 epochs on this dataset
By default, you'll be charged for ~45252 tokens
Estimated training cost ~$0.362016


In [18]:
with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(testing_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-WFUQV5XxSjXm2sY5v3UC5YNi
Validation file ID: file-dZA6oFd3DMt0W2hElIpNHZNy


# Fine Tuning

In [19]:
# Only Run this cell when wanting to create a new fine-tuning job, otherwise you will be paying to redo work

#Uncomment the below code when wanting to run a new fine-tuning job
# response = client.fine_tuning.jobs.create(
#     training_file=training_file_id,
#     validation_file=validation_file_id,
#     model="gpt-3.5-turbo",
#     suffix="interview_tuned_v1",
# )

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-IYAgMaFjULJFM1aJrBJIp0eq
Status: validating_files


In [21]:
#Check Job Status
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


Job ID: ftjob-IYAgMaFjULJFM1aJrBJIp0eq
Status: succeeded
Trained Tokens: 45072


In [22]:
#Track Fine-Tuning Endpoints
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 73/90: training loss=1.50, validation loss=1.52
Step 74/90: training loss=1.24, validation loss=1.24
Step 75/90: training loss=1.46, validation loss=1.76
Step 76/90: training loss=1.24, validation loss=1.62
Step 77/90: training loss=1.53, validation loss=1.72
Step 78/90: training loss=1.81, validation loss=2.17
Step 79/90: training loss=1.32, validation loss=1.52
Step 80/90: training loss=1.49, validation loss=1.32
Step 81/90: training loss=1.86, validation loss=1.87
Step 82/90: training loss=1.87, validation loss=1.85
Step 83/90: training loss=1.62, validation loss=1.80
Step 84/90: training loss=1.81, validation loss=1.81
Step 85/90: training loss=1.75, validation loss=1.45
Step 86/90: training loss=1.71, validation loss=2.28
Step 87/90: training loss=2.17, validation loss=1.48
Step 88/90: training loss=2.28, validation loss=1.51
Step 89/90: training loss=1.45, validation loss=1.22
Step 90/90: training loss=1.51, validation loss=1.74
New fine-tuned model created: ft:gpt-3.5-turbo

In [23]:
# When job is done, run to fets fine-tuned model id
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:interview-tuned-v1:98u7HS8r


## Fine-Tuned Model Testing

In [24]:
df_interview_dev = pd.read_csv("https://raw.githubusercontent.com/UrologyUnbound/SIOP_ML_2024_Discord/main/data/dev/interview_val_public.csv")

In [25]:
test_row = df_interview_dev.iloc[2]

test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': '\n'
             'Your task is to analyze a dataset of interview questions and '
             'responses. Based on the content, tone, and details of the '
             'provided answers, your task is to \n'
             ' generate a plausible answer for the fourth interview '
             'question.\n',
  'role': 'system'},
 {'content': 'Question: Everyone faces moments of low motivation. Can you '
             "share a story about how you've handled such a moment, including "
             'the steps you took to regain your drive and enthusiasm?\n'
             'Answer: Well currenly i am in a big fight with my roomates over '
             'a cat so one thing I did was take a step back went to take a nap '
             'and listen to feel good music\n'
             'Question: Going above and beyond is a commendable trait. Can you '
             'share a story of a time when you did more than was required, and '
             'discuss the impact it had on the project’s outc

In [26]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

I was working with a group of people who were all very strong willed and had a lot of ideas. I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was not a leader and I was the only one who was 